In [2]:
import cv2
import numpy as np
import csv
import sys
import os

Process images from blender and store the data (combined with the correct position data from the text file) in a csv file

In [3]:
# Function to display progress in the console
def update_progress(progress):
    bar_length = 50  # Total length of the progress bar
    filled_length = int(bar_length * progress / 100)
    bar = '#' * filled_length + '_' * (bar_length - filled_length)
    sys.stdout.write('\r[{0}] {1}%'.format(bar, progress))
    sys.stdout.flush()

Each image represents a unique possible position of the game object, in the database we describe each image with 5 values: center_pixel_x, center_pixel_y, rect_width, rect_height, angle

In [37]:
# Function to find red regions and extract bounding rectangle
def find_oriented_bounding_rect(image):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([0, 20, 25])
    upper_red = np.array([30, 255, 255])
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Find contours
    contours, _ = cv2.findContours(red_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)

    # Get the minimum area rectangle
    oriented_rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(oriented_rect)
    box = np.intp(box)

    # Draw the bounding box
    cv2.drawContours(image, [box], 0, (0, 0, 255), 2)
    cv2.imshow("Frame", image)
    cv2.waitKey(1)

    return oriented_rect

Load data from Blender

In [ ]:
# Image folder path and CSV file path
images_folder = 'D:/FRC/Programming/LearningProjects/CoralPoseEstimation-Old/Images' #! Image folder is not included due to size constraints of github repository
text_file_path = '../Data/2025-Coral/PositionData.txt'
csv_file_path = '../Data/2025-Coral/FullData.csv'

In [20]:
# Count .png files in the folder
total_iterations = len([file for file in os.listdir(images_folder) if file.endswith('.jpg')])
print(f"Total images found: {total_iterations}")
# Get a sorted list of image filenames in the folder
image_files = range(0, total_iterations)


position_data = []
with open(text_file_path, 'r') as file:
    for line in file:
        x, y, a = line.strip().split(',')
        
        position_data.append([float(x), float(y), float(a)])

if (len(position_data) != total_iterations):
    print(f"Warning: Number of position data points ({len(position_data)}) does not match number of images ({total_iterations}).")

Total images found: 198936


Process Data from Blender

In [44]:
targetRes = (1280, 720)

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image', 'Center_X', 'Center_Y', 'Width', 'Height', 'Angle', 'x_position', 'y_position', 'angle'])

    # Loop through all files in the folder
    for filename in image_files:
        # Read image
        image = cv2.imread(os.path.join(images_folder, ("render_"+ str(filename)+ ".jpg")))
        
        image = cv2.resize(image, targetRes, interpolation=cv2.INTER_LINEAR)

        # Find red rectangles
        oriented_rect = find_oriented_bounding_rect(image)

        # Extract rectangle parameters
        (center_x, center_y), (width, height), angle = oriented_rect

        csv_writer.writerow([filename, np.intp(center_x), np.intp(center_y), np.intp(width), np.intp(height), np.intp(angle), position_data[filename][0], position_data[filename][1], position_data[filename][2]])
        
        # Update progress
        progress = int(100 * (filename + 1) / total_iterations)
        update_progress(progress)

cv2.destroyAllWindows()

[##################################################] 100%

In [39]:
cv2.destroyAllWindows()